## 从CheckPoint文件中恢复模型

In [1]:
import urllib.request
import os

data_url="http://biostat.mc.vanderbilt.edu/wiki/put/Main/DataSets/titanic3.xls"
data_file_path="/Users/lumin/Documents/dataset/Titanic_data/titanic3.xls"

if not os.path.isfile(data_file_path):
    result=urllib.request.urlretrieve(data_url, data_file_path)
    print('downloaded:', result)
else:
    print(data_file_path, 'data file alreday exists.')

/Users/lumin/Documents/dataset/Titanic_data/titanic3.xls data file alreday exists.


In [2]:
import numpy
import pandas as pd
#读取数据文件，结果为DataFrame格式
df_data = pd.read_excel(data_file_path)

In [3]:
selected_cols = ['survived','name','pclass','sex','age','sibsp','parch','fare','embarked']
selected_df_data = df_data[selected_cols]

In [4]:
from sklearn import preprocessing
def prepare_data(df_data):
    df = df_data.drop(['name'], axis=1)
    age_mean = df['age'].mean()
    df['age'] = df['age'].fillna(age_mean)
    fare_mean = df['fare'].mean()
    df['fare'] = df['fare'].mean()
    df['fare'] = df['fare'].fillna(fare_mean)
    df['sex'] = df['sex'].map({'female':0, 'male':1}).astype(int)
    df['embarked'] = df['embarked'].fillna('S')
    df['embarked'] = df['embarked'].map({'C':0, 'Q':1, 'S':2}).astype(int)
    
    ndarray_data = df.values
    
    features = ndarray_data[:,1:]
    label = ndarray_data[:,0]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    norm_features = minmax_scale.fit_transform(features)
    
    return norm_features, label

In [5]:
shuffled_df_data = selected_df_data.sample(frac=1)

In [6]:
x_data,y_data=prepare_data(shuffled_df_data)

In [7]:
train_size = int(len(x_data)*0.8)

x_train = x_data[:train_size]
y_train = y_data[:train_size]

x_test = x_data[train_size:]
y_test = y_data[train_size:]

In [8]:
import tensorflow as tf
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=64,
                                input_dim=7,
                                use_bias=True,
                                kernel_initializer='uniform',
                                bias_initializer='zeros',
                                activation='relu'))
                                
model.add(tf.keras.layers.Dense(units=32, activation='sigmoid'))

model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.003),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
logdir = '/Users/lumin/Documents/dataset/Titanic_data/logs'
checkpoint_path = '/Users/lumin/Documents/dataset/Titanic_data/checkpoint/Titanic.{epoch:02d}-{val_loss:.2f}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/Users/lumin/Documents/dataset/Titanic_data/checkpoint/Titanic.100-0.50.ckpt'

In [13]:
model.load_weights(latest)

## 评估模型

In [14]:
loss,acc = model.evaluate(x_test, y_test)
print("Restored model, accuracy:{:5.2f}%".format(100*acc))

262/262 [==============================] - 0s 115us/sample - loss: 0.4526 - acc: 0.7824
Restored model, accuracy:78.24%


In [15]:
model.loss

'binary_crossentropy'

In [16]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(7, 64) dtype=float32>,
 <tf.Variable 'dense/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'dense_1/kernel:0' shape=(64, 32) dtype=float32>,
 <tf.Variable 'dense_1/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'dense_2/kernel:0' shape=(32, 1) dtype=float32>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32>]